In [1]:
import pandas as pd

data = pd.read_csv("spamdata.csv",encoding = "latin-1")

In [3]:
data.head(20)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [6]:
import string
punctuations = string.punctuation

from nltk.corpus import stopwords

stopwords_list = stopwords.words("english")

from nltk.stem.wordnet import WordNetLemmatizer

lem = WordNetLemmatizer()


def _clean(text):
    cleaned_text = text.lower()
    
    cleaned_text = "".join(c for c in text if c not in punctuations)
    
    words = cleaned_text.split()
    
    words = [w for w in words if w not in stopwords_list]
    
    words = [lem.lemmatize(word,"v") for word in words]
    words = [lem.lemmatize(word,"n") for word in words]
    
    cleaned_text =" ".join(words)
    
    return cleaned_text



'play game today'

In [7]:
data["cleaned"] = data["text"].apply(_clean)

In [13]:
#feature enginering
#meta features


data["word_count"] = data["text"].apply(lambda x : len(x.split()))
data["word_count_cleaned"] = data["cleaned"].apply(lambda x : len(x.split()))

data["char_count"] = data["text"].apply(lambda x : len(x))
data["char_count_without_spaces"] = data["text"].apply(lambda x : len(x.replace(" ","")))

data["num_digit"] = data["text"].apply(lambda x : sum([1 if w.isdigit() else 0 for w in x.split()]))


In [14]:
data.head()


,label,text,cleaned,word_count,word_count_cleaned,char_count,char_count_without_spaces,num_digit
0,ham,"Go until jurong point, crazy.. Available only ...",Go jurong point crazy Available bugis n great ...,20,16,111,92,0
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,6,6,29,24,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry 2 wkly comp win FA Cup final tkts 2...,28,23,155,128,2
3,ham,U dun say so early hor... U c already then say...,U dun say early hor U c already say,11,9,49,39,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think go usf live around though,13,9,61,49,0


In [19]:
pos_dic={"noun":["NNP","NN","NNS","NNPS"],"verb":["VBZ","VB","VBD","VBN","VBG"]}

import nltk

def pos_check(text,family):
    tags = nltk.pos_tag(nltk.word_tokenize(text))
    count=0
    
    for tag in tags:
        tag = tag[1]
        if tag in pos_dic[family]:
            count+=1
    return count
    
#pos_check("i am playing in the ground","noun") 

In [21]:
data["noun_count"] = data["text"].apply(lambda x : pos_check(x,"noun"))
data["verb_count"] = data["text"].apply(lambda x : pos_check(x,"verb"))

In [23]:
data.head()

,label,text,cleaned,word_count,word_count_cleaned,char_count,char_count_without_spaces,num_digit,noun_count,verb_count
0,ham,"Go until jurong point, crazy.. Available only ...",Go jurong point crazy Available bugis n great ...,20,16,111,92,0,10,1
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,6,6,29,24,0,4,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry 2 wkly comp win FA Cup final tkts 2...,28,23,155,128,2,13,3
3,ham,U dun say so early hor... U c already then say...,U dun say early hor U c already say,11,9,49,39,0,3,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think go usf live around though,13,9,61,49,0,1,4


In [39]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer

cvz = CountVectorizer()
cvz.fit(data["cleaned"].values)

count_vectors = cvz.transform(data["cleaned"].values)

In [40]:
count_vectors

<5572x8590 sparse matrix of type '<class 'numpy.int64'>'
	with 51048 stored elements in Compressed Sparse Row format>

In [50]:
word_tfidf = TfidfVectorizer(max_features=500)
word_tfidf.fit(data["cleaned"].values)

word_vectors_tfidf = word_tfidf.transform(data["cleaned"].values)

In [52]:
ngram_tfidf = TfidfVectorizer(max_features=500,ngram_range=(1,2))
ngram_tfidf.fit(data["cleaned"].values)

ngram_tfidf = ngram_tfidf.transform(data["cleaned"].values)

In [53]:
char_tfidf = TfidfVectorizer(max_features=500,analyzer="char")
char_tfidf.fit(data["cleaned"].values)

char_tfidf = char_tfidf.transform(data["cleaned"].values)

In [46]:
tfidf = dict(zip(word_tfidf.get_feature_names(),word_tfidf.idf_))
tfidf_idf = pd.DataFrame(columns=["word_tfidf"]).from_dict(tfidf,orient="index")
tfidf_idf.columns = ["word_tfidf"]


In [54]:
from scipy.sparse import hstack, csr_matrix

meta_features = ['word_count', 'word_count_cleaned',
       'char_count', 'char_count_without_spaces', 'num_digit', 'noun_count',
       'verb_count']

feature_set1 = data[meta_features]

train = hstack([word_vectors_tfidf,csr_matrix(feature_set1)],"csr")
train

<5572x507 sparse matrix of type '<class 'numpy.float64'>'
	with 64155 stored elements in Compressed Sparse Row format>

In [56]:
from sklearn.preprocessing import LabelEncoder

target = data["label"].values
target = LabelEncoder().fit_transform(target)

In [58]:
target

array([0, 0, 1, ..., 0, 0, 0])

In [60]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(train, target)

In [62]:
train_x.shape

(4179, 507)

In [64]:
val_x.shape

(1393, 507)

In [66]:
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import ensemble
from sklearn.metrics import accuracy_score

In [68]:
model = naive_bayes.MultinomialNB()
model.fit(train_x,train_y)
preds = model.predict(val_x)
accuracy_score(preds,val_y)

0.9755922469490309

In [71]:
model = LogisticRegression()
model.fit(train_x,train_y)
preds = model.predict(val_x)
accuracy_score(preds,val_y)

/Users/suryavikram/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9856424982053122

In [72]:
model = svm.SVC()
model.fit(train_x,train_y)
preds = model.predict(val_x)
accuracy_score(preds,val_y)

/Users/suryavikram/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9490308686288585

In [74]:
model = ensemble.ExtraTreesClassifier()
model.fit(train_x,train_y)
preds = model.predict(val_x)
accuracy_score(preds,val_y)

/Users/suryavikram/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9770279971284996

In [ ]:
import numpy as np

embeddings_index={}